In [1]:
import torch
from PIL import Image
from torchvision import transforms
import numpy as np


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
noisy = Image.open("/content/drive/MyDrive/R2R_ML/an_image/test/Sony_4-5_125_3200_plant_13_real.JPG")
transform = transforms.ToTensor()
n = transform(noisy) #of shape (3,512,512)


In [4]:
#Generating patches
kh, kw = 8, 8  # kernel size
dh, dw = 3, 3  # stride

patches = n.unfold(1, kh, dh).unfold(2, kw, dw) #of shape (3,h_patch_patch,w_patch_num,kh,kw)
patches = patches.permute((2,1,0,3,4)).reshape(-1,3,kh, kw) #of shape (num_patches,3,kh,kw)
patches = patches.reshape((-1,8*8*3)).permute((1,0))  #of shape (dim,num_patches)

mu = patches.mean(axis=1, keepdims=True)  # of shape (dim,1)
X = patches - mu  #of shape (dim,num_patches)
sigma_X = np.matmul(X, X.T) / patches.shape[1] #of shape (dim,dim)

eigen_vals, _ = np.linalg.eigh(sigma_X)
eigen_vals.sort() #of shape (dim)

for i in range(patches.shape[1]-1,-1,-1):
    tau = np.mean(eigen_vals[:i])
    if np.sum(eigen_vals[:i]>tau) == np.sum(eigen_vals[:i]<tau): #Checking if tau is the median
        break

print(np.sqrt(tau)*255)

0.4434824560303241
